# Import Dependencies

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#pip install fredapi 

In [ ]:
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import numpy as np
import datetime
import seaborn as sn
from fredapi import Fred 
fred = Fred(api_key='45071d0d8c02d0f45bf4ea219541507f')
plt.rcParams['figure.figsize'] = [12, 8]
currentYear = datetime.datetime.now().year

russell_path = 'Resources/Returns for AJ.xlsx'
inflation_path = 'Resources/inflation_rates.xlsx'
fed_rate_path = 'Resources/fed-funds-rate-historical-chart.csv'
unemployment = fred.get_series('UNRATE', observation_start='1970-1-1')
housing_starts = fred.get_series('HOUST2FNSA', observation_start='1970-1-1') # seasonally ajusted non-fram employment in thousands 
cci_path = 'Resources/cci.csv'
man_emp = fred.get_series('MANEMP', observation_start='1970-1-1') # seasonally ajusted non-fram employment in thousands 

# Create Initial Dataframes

In [ ]:
russell_df = pd.read_excel(russell_path, header=1)
inflation_df = pd.read_excel(inflation_path, header=0)
fed_rate_df = pd.read_csv(fed_rate_path)
unemployment_df = unemployment.reset_index().rename(columns = {'index': 'date', 0 :'UNRATE'})
housing_starts_df = housing_starts.reset_index().rename(columns = {'index': 'date', 0 :'Housing Starts'})
cci_df = pd.read_csv(cci_path)
man_emp_df = man_emp.reset_index().rename(columns = {'index': 'date', 0 :'MANEMP'})

In [ ]:
# Creat time_period variable for future calculations

time_period = len(russell_df)/12

# Defining Formulas

In [ ]:
# Will correct any dates converted incorrectly (> 2022)
def datefix(x):
    if x.year > currentYear:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)

#Will Calculate quarter based on month number
def quartercalc(month):
    quarter = 0
    if month in [1,2,3]:
        quarter = 1
    if month in [4,5,6]:
        quarter = 2
    if month in [7,8,9]:
        quarter = 3
    if month in [10,11,12]:
        quarter = 4
    return(quarter)

# Will convert 3 letter month codes to number. (Jan = 1, Dec = 12)
def monthconvert(month):
    datetime_object = datetime.datetime.strptime(month, "%b")
    month_number = datetime_object.month
    return(month_number)

# Will convert percentage values to decimal

def percentageconvert(value):
    return (value / 100)

#Will add one to decimal percentage to allow for compounding. 

def percentagecompound(value):
    return value + 1

#Will subtract one from value created by percentagecompound.

def percentreverse(value):
    return (value-1)

#Will multiply percentage value times 100. 

def hundred(value):
    return(value)*100

#Will determine annualized returns from monthly values. 

def annualfrommonthlyreturns(value):
    return (((value + 1)**12) - 1)

#Will determine annualized returns from monthly values. 

def annualfrommonthlystd(value):
    return value * (12**1/2)

#Will determine annualized returns from monthly values. 

def annualfromquarterlystd(value):
    return value * (4**1/2)


#def highlightTrades(value):
 #   if 'Trade' in value:
  #      return ['background-color: Green']
 
# Swaps favor for portfolio names.     
def portswitch(value):
    if value == 'Russell 1000 Core':
        return 'Core Weighted'
    if value == 'Russell 1000 Growth':
        return 'Growth Weighted'
    if value == 'Russell 1000 Value':
        return 'Value Weighted'
    
#Assign Buy / Sell directions
def advtradeflag(value):
    if value > 0: 
        return(f'Buy: {round(value*100,4)}%')
    if value < 0: 
        return(f'Sell: {round(value*100,4)}%')
    if value == 0: 
        return(f'Do Not Trade')

# Data Cleaning

In [ ]:
#Creates Month, Day, Year Columns, and converts to numbers. 
unemployment_df['Month'] = unemployment_df['date'].astype(str).str.split('-').str[1]
unemployment_df['Day'] = unemployment_df['date'].astype(str).str.split('-').str[2]
unemployment_df['YEAR'] = unemployment_df['date'].astype(str).str.split('-').str[0]
unemployment_df[['Month', 'YEAR', 'Day']] = unemployment_df[['Month', 'YEAR', 'Day']].apply(pd.to_numeric)

# Adds century to YY
#man_emp_df.loc[man_emp_df['YEAR'] + 2000 > currentYear, "YEAR"] = man_emp_df['YEAR'] + 1900
#man_emp_df.loc[man_emp_df['YEAR'] + 2000 <= currentYear, "YEAR"] = man_emp_df['YEAR'] + 2000

#Calculates Change and Percentage Change
unemployment_df['Unemployment Change'] =  unemployment_df['UNRATE'].shift(-1) - unemployment_df['UNRATE']
unemployment_df['Unemployment Change (Pct)'] = (unemployment_df['Unemployment Change'] / unemployment_df['UNRATE'])*100

#Sets index and removes unwanted columns / rows. 
unemployment_df = unemployment_df.set_index('YEAR')
unemployment_df = unemployment_df.drop(columns = ['date','Day'])
unemployment_df = unemployment_df[(unemployment_df.index > 1978) & (unemployment_df.index < 2016)]

unemployment_df

In [ ]:
# Parses Date numbers from date, converts them to numbers. 
fed_rate_df['Month'] = fed_rate_df['date'].str.split('/').str[0]
fed_rate_df['Day'] = fed_rate_df['date'].str.split('/').str[1]
fed_rate_df['YEAR'] = fed_rate_df['date'].str.split('/').str[2]
fed_rate_df[['Month', 'YEAR', 'Day']] = fed_rate_df[['Month', 'YEAR', 'Day']].apply(pd.to_numeric)

#converts 'date' to datetime
fed_rate_df['date'] = pd.to_datetime(fed_rate_df['date'])

#adds 1900/2000 to 'YEAR', to update from YY to YYYY, and sets as index
fed_rate_df.loc[fed_rate_df['YEAR'] + 2000 > currentYear, "YEAR"] = fed_rate_df['YEAR'] + 1900
fed_rate_df.loc[fed_rate_df['YEAR'] + 2000 <= currentYear, "YEAR"] = fed_rate_df['YEAR'] + 2000
fed_rate_df = fed_rate_df.set_index('YEAR')

#Applies date fix 
fed_rate_df['date'] = fed_rate_df['date'].apply(datefix)

#fed_rate_df

In [ ]:
# Creates list (DF) of values to keep. 

fed_rate_delete = fed_rate_df.copy()
fed_rate_delete = fed_rate_delete.drop([' value'], axis = 1)
fed_rate_delete = fed_rate_delete.groupby(['Month', 'YEAR'], as_index=False, sort = True).max()
fed_rate_delete = fed_rate_delete.drop(columns =['Month','Day'])

#fed_rate_delete

In [ ]:
# Finds dates that are in the fed_rate_delete list and created new dataframe
fed_rate_df_clean = fed_rate_df.loc[fed_rate_df['date'].isin(fed_rate_delete['date'])].copy()

# Calculated change and pct change in Fed Rate
fed_rate_df_clean['Fed Rate Change'] =  fed_rate_df_clean[' value'].shift(-1) - fed_rate_df_clean[' value']
fed_rate_df_clean['Fed Rate Change (Pct)'] = (fed_rate_df_clean['Fed Rate Change'] / fed_rate_df_clean[' value'])*100

# Removes years outside of scope, and deletes unwanted columns
fed_rate_df_clean = fed_rate_df_clean[(fed_rate_df_clean.index > 1978) & (fed_rate_df_clean.index < 2016)]
fed_rate_df_clean = fed_rate_df_clean.drop(columns = ['Day'])

# Calculates Quarter from Month
fed_rate_df_clean['Quarter'] = fed_rate_df_clean['Month'].apply(quartercalc)

#fed_rate_df_clean

In [ ]:
#Transposes dataset, drops unused columns, and renames remaining. 
inflation_df2 = inflation_df.set_index('YEAR')
inflation_df2 = inflation_df2.drop(columns = 'AVE')
inflation_df2 = inflation_df2.reset_index().melt(id_vars=['YEAR']).set_index(['YEAR'])
inflation_df2 = inflation_df2.rename(columns = {'variable': 'Month', 'value': 'Inflation Rate'})

#inflation_df2.head()

In [ ]:
# Converts month codes to numbers. 
inflation_df2['Month'] = inflation_df2.apply(lambda row: monthconvert(row['Month']),axis=1)

#Sorts DF by year and month
inflation_df2 = inflation_df2.sort_values(by=['YEAR', "Month"])

#Calculates change in inflation and percentage change in inflation. 

inflation_df2['Inflation Change'] =  inflation_df2['Inflation Rate'].shift(-1) - inflation_df2['Inflation Rate']
inflation_df2['Inflation Change (Pct)'] = (inflation_df2['Inflation Change'] / inflation_df2['Inflation Rate'])*100

# Removes Years out of scope. 
inflation_df2 = inflation_df2[(inflation_df2.index > 1978) & (inflation_df2.index < 2016)]

#inflation_df2

In [ ]:
# Creates Month and Year columns based on Date column, and converts them to numbers. 
russell_df['Month'] = russell_df['Date'].str.split('/').str[0]
russell_df['YEAR'] = russell_df['Date'].str.split('/').str[1]
russell_df[['Month', 'YEAR']] = russell_df[['Month', 'YEAR']].apply(pd.to_numeric)

# Converts YEAR column to YYYY from YY. 
russell_df.loc[russell_df['YEAR'] + 2000 > currentYear, "YEAR"] = russell_df['YEAR'] + 1900
russell_df.loc[russell_df['YEAR'] + 2000 <= currentYear, "YEAR"] = russell_df['YEAR'] + 2000

# Remove Unwanted Columns
russell_df = russell_df.drop(columns = ['Unnamed: 0', 'Date'])

#russell_df.head()

# Begin Merging Dataframes total dataset

In [ ]:
# Merge Inflation and Fed Rate Dataframes

market_merge_df = pd.merge(inflation_df2, fed_rate_df_clean, on = ['Month','YEAR'], how = 'left')

#market_merge_df

In [ ]:
# Merger market_merge_df with unemployment. 

market_merge_df = pd.merge(market_merge_df,unemployment_df , on = ['Month','YEAR'], how = 'left')
market_merge_df = market_merge_df.reset_index()

#market_merge_df

In [ ]:
#Declare column headers, drop unused columns, and reorder columns based on list

column_list = ['date', 'Month', 'Quarter','YEAR','Fed Rate Change', 'Fed Rate Change (Pct)','Unemployment Change','Unemployment Change (Pct)','Inflation Change', 'Inflation Change (Pct)']
market_merge_df = market_merge_df.drop(columns = [' value', 'UNRATE', 'Inflation Rate'])
market_merge_df = market_merge_df.reindex(columns = column_list)

#market_merge_df


In [ ]:
# Merger Russell data to market data

market_merge_df = pd.merge(market_merge_df,russell_df , on = ['Month','YEAR'], how = 'left')

#market_merge_df

In [ ]:
# Set Index to Year, Quarter, Month, Date

market_merge_df_indexed = market_merge_df.set_index(['YEAR','Quarter','Month','date'])

#market_merge_df_indexed

# Monthly Market Data w/ Indicator Change

In [ ]:
# Create Monthly Market Data DF w/ changes (no pct changes)
market_merge_df_no_pct = market_merge_df_indexed.drop(columns = ['Fed Rate Change (Pct)', 'Unemployment Change (Pct)','Inflation Change (Pct)']).copy()
market_merge_df_no_pct = market_merge_df_no_pct[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Fed Rate Change','Unemployment Change','Inflation Change']]

#market_merge_df_no_pct


# Splitting Indicator Dataframe to Create Quarterly / Yearly Dataframes.

In [ ]:
# Splitting monthly Indicator data from market dataframe
monthly_indicator_data = market_merge_df_no_pct[['Fed Rate Change', 'Unemployment Change','Inflation Change']].reset_index().groupby(['YEAR','Quarter','Month']).agg({'date':'max','Fed Rate Change': 'sum','Unemployment Change': 'sum','Inflation Change': 'sum'}).reset_index().set_index(['YEAR', 'Quarter','Month', 'date'])

#monthly_indicator_data


# Quarterly

In [ ]:
# Aggregating indicator data quarterly
quarterly_indicator_data = market_merge_df_no_pct[['Fed Rate Change', 'Unemployment Change','Inflation Change']].reset_index().drop(columns = 'Month').groupby(['YEAR','Quarter']).agg({'date':'max','Fed Rate Change': 'sum','Unemployment Change': 'sum','Inflation Change': 'sum'}).reset_index().set_index(['YEAR', 'Quarter', 'date'])

#quarterly_indicator_data

# Yearly

In [ ]:
# Aggregating indicator data, yearly. 

yearly_indicator_data = market_merge_df_no_pct[['Fed Rate Change', 'Unemployment Change','Inflation Change']].reset_index().drop(columns = ['Month','Quarter']).groupby(['YEAR']).agg({'date':'max','Fed Rate Change': 'sum','Unemployment Change': 'sum','Inflation Change': 'sum'}).reset_index().set_index(['YEAR', 'date'])

#yearly_indicator_data

# # Monthly Market Data w/ Indicator Change (Pct)

In [ ]:
# Creating monthly dataframe with pct changes
market_merge_pct_monthly = market_merge_df_indexed.drop(columns = ['Fed Rate Change', 'Unemployment Change','Inflation Change']).reindex(columns = ['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Fed Rate Change (Pct)','Unemployment Change (Pct)','Inflation Change (Pct)']).copy()

#market_merge_pct_monthly

# Monthly Market Data For Compounding

# Splitting Compounding Dataframe to Create Quarterly / Yearly Dataframes. 

In [ ]:
#Removing non percentage columns from dataframe to create compounding dataframe (/100 + 1 to returns and pct changes)
market_merge_df_compound = market_merge_df_indexed.drop(columns = ['Fed Rate Change', 'Unemployment Change','Inflation Change']).copy()
market_merge_df_compound = market_merge_df_compound[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Fed Rate Change (Pct)','Unemployment Change (Pct)','Inflation Change (Pct)']].apply(percentageconvert).apply(percentagecompound)
market_merge_df_compound = market_merge_df_compound.replace([np.inf, -np.inf],1)

market_merge_df_compound


## Quarterly

In [ ]:
# Aggregating russell pct dataframe quarterly
quarterly_strat_data = market_merge_df_compound[['Russell 1000 Core', 'Russell 1000 Growth','Russell 1000 Value']].reset_index().drop(columns = 'Month').groupby(['YEAR','Quarter']).agg({'date':'max','Russell 1000 Core': 'prod','Russell 1000 Growth': 'prod','Russell 1000 Value': 'prod'}).reset_index().set_index(['YEAR', 'Quarter', 'date']).apply(percentreverse)

#quarterly_strat_data

## Yearly

In [ ]:
# Aggregating russell pct dataframe yearly

yearly_strat_data = market_merge_df_compound[['Russell 1000 Core', 'Russell 1000 Growth','Russell 1000 Value']].reset_index().drop(columns = ['Month', 'Quarter']).groupby(['YEAR']).agg({'date':'max','Russell 1000 Core': 'prod','Russell 1000 Growth': 'prod','Russell 1000 Value': 'prod'}).reset_index().set_index(['YEAR','date']).apply(percentreverse).round(6)

#yearly_strat_data


# Completed Data Frames

# Simple

### Monthly

In [ ]:
# Simple, monthly indicator changes + russell returns. 

monthly_indicator_join = monthly_indicator_data.reset_index().drop(columns = ['YEAR', 'Quarter','Month'])
monthly_strat_join = market_merge_df_compound.reset_index().drop(columns = ['YEAR','Quarter','Month','Fed Rate Change (Pct)','Unemployment Change (Pct)', 'Inflation Change (Pct)']).set_index('date').apply(percentreverse)
simple_monthly_md = pd.merge(monthly_strat_join,monthly_indicator_join, how = 'left', on = 'date').set_index('date')

simple_monthly_md


## Quarterly

In [ ]:
# Simple, quarterly indicator changes + russell returns. 

quarterly_indicator_join = quarterly_indicator_data.reset_index().drop(columns = ['YEAR', 'Quarter'])
quarterly_strat_join = quarterly_strat_data.reset_index().drop(columns = ['YEAR', 'Quarter'])
simple_quarterly_md = pd.merge(quarterly_strat_join,quarterly_indicator_join, how = 'left', on = 'date').set_index('date')

simple_quarterly_md


## Yearly

In [ ]:
# Simple, yearly indicator changes + russell returns. 

yearly_indicator_join = yearly_indicator_data.reset_index().drop(columns = ['YEAR'])
yearly_strat_join = yearly_strat_data.reset_index().drop(columns = ['YEAR'])
simple_yearly_md = pd.merge(yearly_strat_join,yearly_indicator_join, how = 'left', on = 'date').set_index('date')

simple_yearly_md

## Relative

### Monthly - Relative

In [ ]:
# Relative, Monthly indicator changes + russell returns. 

relative_monthly_md = market_merge_df_compound.reset_index().drop(columns = ['YEAR','Quarter','Month']).set_index('date').apply(percentreverse)

relative_monthly_md


## Quarterly - Relative

In [ ]:
# Relative, Quarterly indicator changes + russell returns. 

relative_quarterly_md = market_merge_df_compound.reset_index().drop(columns = ['Month']).groupby(['YEAR','Quarter']).agg({'date':'max','Russell 1000 Core': 'prod','Russell 1000 Growth': 'prod','Russell 1000 Value': 'prod','Fed Rate Change (Pct)': 'prod','Unemployment Change (Pct)': 'prod','Inflation Change (Pct)': 'prod'}).reset_index().drop(columns = ['YEAR', 'Quarter']).set_index('date').apply(percentreverse).round(4)

relative_quarterly_md

## Yearly - Relative


In [ ]:
# Relative, Yearly indicator changes + russell returns. 

relative_yearly_md = market_merge_df_compound.reset_index().drop(columns = ['Month','Quarter']).groupby(['YEAR']).agg({'date':'max','Russell 1000 Core': 'prod','Russell 1000 Growth': 'prod','Russell 1000 Value': 'prod','Fed Rate Change (Pct)': 'prod','Unemployment Change (Pct)': 'prod','Inflation Change (Pct)': 'prod'}).reset_index().drop(columns = ['YEAR']).set_index('date').apply(percentreverse).round(6)

relative_yearly_md


# Initial Stats

In [ ]:
stats = simple_monthly_md[['Russell 1000 Core','Russell 1000 Value','Russell 1000 Growth']].describe().T.drop(columns = ['25%','50%','75%','count', 'mean','std']).apply(hundred).round(2).reset_index().rename(columns = {'index':'Strategy'}).set_index('Strategy').copy()
#stats['mean'] = stats['mean'].apply(percentageconvert).apply(annualfrommonthlyreturns).apply(hundred)
#stats['std'] = stats['std'].apply(annualfrommonthlystd)
#stats['Sharpe'] = (stats['mean'] - 2) / stats['std']

#stats


In [ ]:
Initial_Ret = ((relative_quarterly_md[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value']].apply(percentagecompound).prod() ** (1/time_period) - 1)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Return'})
Initial_Std = (relative_quarterly_md[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value']].std().apply(annualfromquarterlystd)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Std'})
Initial_Stats = pd.merge(Initial_Ret, Initial_Std, on='Strategy', how = 'left')
Initial_Stats['Sharpe'] = ((Initial_Stats['Annualized Return'] - 2) / Initial_Stats['Annualized Std']).round(2)
#Initial_Stats = Initial_Stats.set_index('Strategy')
Initial_Stats = pd.merge(Initial_Stats, stats, on='Strategy', how = 'left').set_index('Strategy')
Initial_Stats


In [ ]:
x1 = simple_monthly_md['Russell 1000 Value'].apply(hundred).to_list()
x2 = simple_monthly_md['Russell 1000 Growth'].apply(hundred).to_list()
x3 = simple_monthly_md['Russell 1000 Core'].apply(hundred).to_list()

In [ ]:
normal = np.random.randn(1000)
norm_data = [normal]
norm_label = ['Normal']

hist_data = [x1]
group_labels = ['Value'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size = .5)

#fig2 = ff.create_distplot(norm_data, norm_label, bin_size=0.5)
#fig2.update_layout(fig.layout)

fig.show()

In [ ]:
hist_data = [x2]
group_labels = ['Growth'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size = .5)

#fig2 = ff.create_distplot(hist_data, group_labels, histnorm= '', bin_size=0.5, show_hist=False, show_curve=True, show_rug=False )
#fig2.update_layout(fig.layout)

fig.show()

In [ ]:
hist_data = [x3]
group_labels = ['Core'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size = .5)

#fig2 = ff.create_distplot(hist_data, group_labels, histnorm= '', bin_size=0.5, show_hist=False, show_curve=True, show_rug=False )
#fig2.update_layout(fig.layout)

fig.show()

# Correlation based on Monthly Values (Simple Indicator)

In [ ]:
# Plots monthly simple correlation as heatmap

market_data_corr = simple_monthly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True).set_title('Correlation based on Monthly Values (Simple Indicator)')

plt.show()

## Based on the available monthly data : 
* Core is highly correlated to both Growth and Value, but Value and Growth appear to be able to provide diversification benefits to each other. 
* Growth stocks are expected to outperform during times of rising inflation
* Value stocks can provide a hedge during times of falling inflation. 
* Value stocks should outperform during times of falling unemployment
* Growth stocks will provide a hedge during times of rising unemployment. 
* All strategies behave simlarly to changes in Interest Rates - Will ignore. 

# Correlation based on Quarterly Values (Simple Indicator)

In [ ]:
# Plots quarterly simple correlation as heatmap

market_data_corr = simple_quarterly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True).set_title('Correlation based on Quarterly Values (Simple Indicator)')

plt.show()

## Based on the quarterly monthly data : 
* Core is highly correlated to both Growth and Value, but Value and Growth appear to be able to provide diversification benefits to each other. 
* Growth stocks are expected to outperform during times of rising inflation
* Value stocks can provide a hedge during times of falling inflation. 
* Value stocks should outperform during times of falling unemployment
* Growth stocks will provide a hedge during times of rising unemployment. 
* On a quarterly basis Growth appears to be more positively correlate to Fed Rate Changes, might overweight.  

# Correlation based on Yearly Values (Simple Indicator)

In [ ]:
# Plots yearly simple correlation as heatmap

market_data_corr = simple_yearly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True).set_title('Correlation based on Yearly Values (Simple Indicator)')
plt.show()

## Based on the available yearly data : 
* Core is highly correlated to both Growth and Value, but Value and Growth appear to be able to provide diversification benefits to each other. 
* Growth stocks are expected to outperform during times of rising inflation
* Value stocks can provide a hedge during times of falling inflation. 
* Value stocks should outperform during times of falling unemployment
* Growth stocks will provide a hedge during times of rising unemployment. 
* All strategies behave simlarly to changes in Interest Rates - slight edge to value / core. 

# Correlation based on Monthly Values (Relative Indicators)

In [ ]:
# Plots monthly relative correlation as heatmap

market_data_corr = relative_monthly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True)

plt.show()

# Correlation based on Quarterly Values (Relative Indicators)

In [ ]:
# Plots quarterly relative correlation as heatmap

market_data_corr = relative_quarterly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True)

plt.show()

# Correlation based on Yearly Values (Relative Indicators)

In [ ]:
# Plots monthly relative correlation as heatmap

market_data_corr = relative_yearly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True)

plt.show()


# Momentum Pick #1 (Pure Index)

In [ ]:
# Picks which index to buy based on last months max return, then replaces first value with Russell 1000 Value as no prior month return was available.

momentum = simple_quarterly_md[['Russell 1000 Core','Russell 1000 Value','Russell 1000 Growth']].copy()
momentum['Pick'] = momentum[['Russell 1000 Core','Russell 1000 Value','Russell 1000 Growth']].shift().idxmax(axis=1)
momentum['Pick'] = momentum['Pick'].replace(np.nan, 'Russell 1000 Value')
#pd.set_option("display.max_rows", None, "display.max_columns", None)

momentum

In [ ]:
# Creates trade flag based on if last month's pick = this month's pick. If not Trade flag is created. If so, Hold.

momentum.loc[momentum['Pick'] != momentum['Pick'].shift(), "Trade/Hold"] = "Trade"
momentum = momentum.replace(np.nan, 'X')

momentum

In [ ]:
# Creates row showing the return of the selected strategy based on the pick column. 

idx, col = pd.factorize(momentum['Pick'])
momentum['Selection Return'] = momentum.reindex(columns=col).to_numpy()[np.arange(len(momentum)), idx]

momentum

In [ ]:
momentum_results = momentum[['Russell 1000 Core','Selection Return']].copy()
momentum_results.loc[momentum_results.index[0], 'Core Portfolio Value'] = 100
momentum_results.loc[momentum_results.index[0], 'Selection Portfolio Value'] = 100
momentum_results = momentum_results.copy()
momentum_results.loc[datetime.datetime.strptime('1978-12-31', '%Y-%m-%d').date()] = [0,0, 100,100]  # adding a row
momentum_results = momentum_results.sort_index()
for i in range(momentum_results.index.size - 1):
    momentum_results['Core Portfolio Value'].iloc[i+1] = (momentum_results['Russell 1000 Core'].apply(percentagecompound).iloc[i+1] * momentum_results['Core Portfolio Value'].iloc[i])
    momentum_results['Selection Portfolio Value'].iloc[i+1] = (momentum_results['Selection Return'].apply(percentagecompound).iloc[i+1] * momentum_results['Selection Portfolio Value'].iloc[i])

momentum_results    


In [ ]:
# Plots all 3 portfolio returns

#subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(momentum.apply(hundred).reset_index(), x="date", y=["Russell 1000 Core",'Russell 1000 Value','Russell 1000 Growth'],
            barmode='group',
             height=400, 
            ) 
#subfig.update_layout(title= 'Yearly Returns (Pct)')
#fig.update_traces(line_color='#0000ff', line_width=5)
#fig2.update_traces(yaxis="y2",line_color='#0F0A09')
#subfig.add_traces(fig.data)
#subfig.update_xaxes(nticks=37)
#subfig.update_xaxes(tickangle=90)
#subfig.show()
fig.show()

In [ ]:
#Plots selected portfolio vs Russell 1000 Core over period. 

fig = px.line(momentum_results.reset_index(), x = 'date', y=['Core Portfolio Value','Selection Portfolio Value'], title='Core vs Momentum v1')
fig.show()

In [ ]:
#Shows ending value of $100

momentum_results[['Core Portfolio Value','Selection Portfolio Value']].iloc[-1].round(2).reset_index().rename(columns = {'index':'Portfolio', 0:'Ending Port Val'}).set_index('Portfolio').style.set_caption('Value of $100')

In [ ]:
#Shows Stats of Momentum strategy

momentum_Ret = ((momentum[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].apply(percentagecompound).prod() ** (1/time_period) - 1)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Return'})
momentum_Std = (momentum[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].std().apply(annualfromquarterlystd)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Std'})
momentum_Stats = pd.merge(momentum_Ret, momentum_Std, on='Strategy', how = 'left')
momentum_Stats['Sharpe'] = ((momentum_Stats['Annualized Return'] - 2) / momentum_Stats['Annualized Std']).round(2)
momentum_Stats = momentum_Stats.set_index('Strategy')
momentum_Stats


In [ ]:
#Shows momentum trade data

momentum_trades = momentum[['Russell 1000 Core','Trade/Hold']].groupby('Trade/Hold').count().rename(columns = {'Russell 1000 Core':'Total'})
momentum_trades['Average Yearly Trades'] = momentum_trades['Total'] / time_period
momentum_trades['Average Yearly Trades'] = momentum_trades['Average Yearly Trades'].round(1)
momentum_trades = momentum_trades.drop('X')
momentum_trades


In [ ]:
#Trade schedule

momentum_trade_pivot = momentum[['Trade/Hold','Pick']].reset_index()
momentum_trade_pivot.loc[momentum_trade_pivot['Trade/Hold'] == 'Trade', 'Trade/Hold' ] = momentum_trade_pivot['Trade/Hold'] + " - " + momentum_trade_pivot['Pick']#momentum_2_trade_pivot = momentum_2_trade_pivot.drop(columns = 'Pick')
momentum_trade_pivot = momentum_trade_pivot.drop(columns = 'Pick')
momentum_trade_pivot['Month'] = momentum_trade_pivot['date'].astype(str).str.split('-').str[1]
momentum_trade_pivot['Year'] = momentum_trade_pivot['date'].astype(str).str.split('-').str[0]
momentum_trade_pivot = momentum_trade_pivot.drop(columns = 'date').pivot(index = 'Year', columns = 'Month').rename(columns = {'03': 'Qtr 1','06': 'Qtr 2','09': 'Qtr 3','12': 'Qtr 4'}).style.apply(lambda x: ["background: green" if 'Trade' in value else "" for value in x], axis = 1)
momentum_trade_pivot


## Notes
Picking solely based on last quarter's winner does not appear to be a winning strategy over the time period. 

# Momentum Pick #2 (Weighted Index)

I will now create weighted portfolios to allow for diversification, and test if it is possible to beat the Russell 1000 Core. 

In [ ]:
#Create returns data based on chosen portfolio weights. 

momentum_weighted = momentum.drop(columns = 'Selection Return').copy()
momentum_weighted['Value Weighted Return'] = momentum_weighted['Russell 1000 Core'] * .02 + momentum_weighted['Russell 1000 Value'] * .97 + momentum_weighted['Russell 1000 Growth'] * .01
momentum_weighted['Growth Weighted Return'] = momentum_weighted['Russell 1000 Core'] * .1 + momentum_weighted['Russell 1000 Growth'] * .2 + momentum_weighted['Russell 1000 Value'] * .7 
momentum_weighted['Core Weighted Return'] = momentum_weighted['Russell 1000 Core'] * .2 + momentum_weighted['Russell 1000 Value'] * .8 + momentum_weighted['Russell 1000 Growth'] * .0

momentum_weighted


In [ ]:
#Find return for selected portfolio

momentum_weighted.loc[momentum_weighted['Pick'] == 'Russell 1000 Core', "Selection Return"] =  momentum_weighted['Russell 1000 Core']
momentum_weighted.loc[momentum_weighted['Pick'] == 'Russell 1000 Growth', "Selection Return"] =  momentum_weighted['Growth Weighted Return']
momentum_weighted.loc[momentum_weighted['Pick'] == 'Russell 1000 Value', "Selection Return"] =  momentum_weighted['Value Weighted Return']
momentum_weighted['Pick'] = momentum_weighted['Pick'].apply(portswitch)
momentum_weighted


In [ ]:
#Creates result table, finding value of $100

momentum_weighted_results = momentum_weighted[['Russell 1000 Core','Selection Return']].copy()
momentum_weighted_results.loc[momentum_weighted_results.index[0], 'Core Portfolio Value'] = 100
momentum_weighted_results.loc[momentum_weighted_results.index[0], 'Selection Portfolio Value'] = 100
momentum_weighted_results2 = momentum_weighted_results.copy()
momentum_weighted_results2.loc[datetime.datetime.strptime('1978-12-31', '%Y-%m-%d').date()] = [0,0, 100,100]  # adding a row
momentum_weighted_results2 = momentum_weighted_results2.sort_index()
for i in range(momentum_weighted_results2.index.size - 1):
    momentum_weighted_results2['Core Portfolio Value'].iloc[i+1] = (momentum_weighted_results2['Russell 1000 Core'].apply(percentagecompound).iloc[i+1] * momentum_weighted_results2['Core Portfolio Value'].iloc[i])
    momentum_weighted_results2['Selection Portfolio Value'].iloc[i+1] = (momentum_weighted_results2['Selection Return'].apply(percentagecompound).iloc[i+1] * momentum_weighted_results2['Selection Portfolio Value'].iloc[i])

momentum_weighted_results2    


In [ ]:
#creates list used to plot trade lines. 

lines_test = momentum_weighted.reset_index()[['date','Trade/Hold']].set_index('date').loc[momentum_weighted['Trade/Hold'] == 'Trade'].drop(columns = 'Trade/Hold').reset_index()
lines_test['date'] = lines_test['date'].astype(str)
lines = lines_test['date'].values


In [ ]:
#plots value of $100 with trade lines. 

fig = px.line(momentum_weighted_results2.reset_index(), x = 'date', y=['Core Portfolio Value','Selection Portfolio Value'], title='Core vs Momentum v2')

for i in range(len(lines)):
    fig.add_vline(x=lines[i], line_width=1, line_dash="solid", line_color="black")

fig.update_xaxes(nticks=30)
fig.update_xaxes(tickangle=45)

fig.show()


In [ ]:
#Plots selected portfolio vs Russell 1000 Core over period. 

fig = px.line(momentum_weighted_results2.reset_index(), x = 'date', y=['Core Portfolio Value','Selection Portfolio Value'], title='Core vs Momentum v2')
fig.show()

In [ ]:
#Value of $100

momentum_weighted_results2[['Core Portfolio Value','Selection Portfolio Value']].iloc[-1].round(2).reset_index().rename(columns = {'index':'Portfolio', 0:'Ending Port Val'}).set_index('Portfolio').style.set_caption('Value of $100')

In [ ]:
#momentum_weighted_trade data 

momentum_weighted_trades = momentum_weighted[['Russell 1000 Core','Trade/Hold']].groupby('Trade/Hold').count().rename(columns = {'Russell 1000 Core':'Total'})
momentum_weighted_trades['Average Yearly Trades'] = momentum_weighted_trades['Total'] / time_period
momentum_weighted_trades['Average Yearly Trades'] = momentum_weighted_trades['Average Yearly Trades'].round(1)
momentum_weighted_trades = momentum_weighted_trades.drop('X')
momentum_weighted_trades


In [ ]:
#Generates stats for momentum weighted strat

momentum_weighted_Ret = ((momentum_weighted[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].apply(percentagecompound).prod() ** (1/time_period) - 1)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Return'})
momentum_weighted_Std = (momentum_weighted[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].std().apply(annualfromquarterlystd)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Std'})
momentum_weighted_Stats = pd.merge(momentum_weighted_Ret, momentum_weighted_Std, on='Strategy', how = 'left')
momentum_weighted_Stats['Sharpe'] = ((momentum_weighted_Stats['Annualized Return'] - 2) / momentum_weighted_Stats['Annualized Std']).round(2)
momentum_weighted_Stats = momentum_weighted_Stats.set_index('Strategy')
momentum_weighted_Stats

In [ ]:
#Creates trade schedule 

momentum_2_trade_pivot = momentum_weighted[['Trade/Hold','Pick']].reset_index()
momentum_2_trade_pivot.loc[momentum_2_trade_pivot['Trade/Hold'] == 'Trade', 'Trade/Hold' ] = momentum_2_trade_pivot['Trade/Hold'] + " - " + momentum_2_trade_pivot['Pick']#momentum_2_trade_pivot = momentum_2_trade_pivot.drop(columns = 'Pick')
momentum_2_trade_pivot = momentum_2_trade_pivot.drop(columns = 'Pick')
momentum_2_trade_pivot['Month'] = momentum_2_trade_pivot['date'].astype(str).str.split('-').str[1]
momentum_2_trade_pivot['Year'] = momentum_2_trade_pivot['date'].astype(str).str.split('-').str[0]
momentum_2_trade_pivot = momentum_2_trade_pivot.drop(columns = 'date').pivot(index = 'Year', columns = 'Month').rename(columns = {'03': 'Qtr 1','06': 'Qtr 2','09': 'Qtr 3','12': 'Qtr 4'}).style.apply(lambda x: ["background: green" if 'Trade' in value else "" for value in x], axis = 1)
momentum_2_trade_pivot

# Efficient Frontier

In [ ]:
# Determines annualized portfolio performance based on monthly returns. 

def portfolio_annualized_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *12
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(12)
    return std, returns

#Loops through creating randomly weighted portfolios

def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(3)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualized_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

In [ ]:
# assigns values for simulated EF 

table = simple_monthly_md[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value']].apply(hundred)
returns = simple_monthly_md[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value']].apply(hundred)
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_portfolios = 25000
risk_free_rate = 0

In [ ]:
# Prints result and plot Efficient Frontier. 

def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=table.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    
    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx],index=table.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualized Return:", round(rp,2))
    print ("Annualized Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualized Return:", round(rp_min,2))
    print ("Annualized Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualized volatility')
    plt.ylabel('annualized returns')
    plt.legend(labelspacing=0.8)

In [ ]:
display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)


# Indicator Pick #1


In [ ]:
# Copies quarterly market data in factor pick dataframe. 
indicator_pick = simple_quarterly_md.copy()

In [ ]:
# Assigns values to "Flags" used to determine which portfolio to use. 
# 1 = Value Weighted
# - 1 = Growth Weighted
# 0 = uncertain

indicator_pick.loc[(indicator_pick['Fed Rate Change']<0),'Fed Flag'] = 1
indicator_pick.loc[(indicator_pick['Fed Rate Change']>0),'Fed Flag'] = -1
indicator_pick.loc[(indicator_pick['Fed Rate Change']==0),'Fed Flag'] = 0
indicator_pick.loc[(indicator_pick['Unemployment Change']<0),'Unemployment Flag'] = 1
indicator_pick.loc[(indicator_pick['Unemployment Change']>0),'Unemployment Flag'] = -1
indicator_pick.loc[(indicator_pick['Unemployment Change']==0),'Unemployment Flag'] = 0
indicator_pick.loc[(indicator_pick['Inflation Change']<0),'Inflation Flag'] = 1
indicator_pick.loc[(indicator_pick['Inflation Change']>0),'Inflation Flag'] = -1
indicator_pick.loc[(indicator_pick['Inflation Change']==0),'Inflation Flag'] = 0

In [ ]:
#Sums Flags to create a prediction
#Positive numbers and 0s will use the Value weighted portfolio.
#Negative numbers will use growth weighted. 

indicator_pick['Flag_Sum'] = indicator_pick['Fed Flag'] + indicator_pick['Unemployment Flag'] + indicator_pick['Inflation Flag']
indicator_pick


In [ ]:
# Assigns favored portfolios based on Flag_Sum column of previous month. 

indicator_pick.loc[(indicator_pick['Flag_Sum'].shift() >0),'Favored'] = 'Russell 1000 Value'
indicator_pick.loc[(indicator_pick['Flag_Sum'].shift() ==0),'Favored'] = 'Russell 1000 Value'
indicator_pick.loc[(indicator_pick['Flag_Sum'].shift() <0),'Favored'] = 'Russell 1000 Growth'
indicator_pick = indicator_pick.replace(np.nan, 'Russell 1000 Value')
indicator_pick

In [ ]:
# Checks to see which portfolio was held last quarter and determines if a Trade is necessary. 

indicator_pick.loc[indicator_pick['Favored'] != indicator_pick['Favored'].shift(), "Trade/Hold"] = "Trade"
indicator_pick = indicator_pick.replace(np.nan, 'X')

indicator_pick

In [ ]:
# assigns portfolio weights 

indicator_pick['Value Weighted Return'] = indicator_pick['Russell 1000 Core'] * .02 + indicator_pick['Russell 1000 Value'] * .97 + indicator_pick['Russell 1000 Growth'] * .01
indicator_pick['Growth Weighted Return'] = indicator_pick['Russell 1000 Core'] * .1 + indicator_pick['Russell 1000 Growth'] * .2 + indicator_pick['Russell 1000 Value'] * .7 
indicator_pick['Core Weighted Return'] = indicator_pick['Russell 1000 Core'] * .2 + indicator_pick['Russell 1000 Value'] * .8 + indicator_pick['Russell 1000 Growth'] * .0

# Creates columns with performance of all 3 strategies

indicator_pick.loc[indicator_pick['Favored'] == 'Russell 1000 Core', "Selection Return"] =  indicator_pick['Value Weighted Return']
indicator_pick.loc[indicator_pick['Favored'] == 'Russell 1000 Growth', "Selection Return"] =  indicator_pick['Growth Weighted Return']
indicator_pick.loc[indicator_pick['Favored'] == 'Russell 1000 Value', "Selection Return"] =  indicator_pick['Value Weighted Return']

indicator_pick['Favored'] = indicator_pick['Favored'].apply(portswitch)

indicator_pick


In [ ]:
# Creates Result dataframe and calculates value of $100

indicator1_results = indicator_pick[['Russell 1000 Core','Selection Return']].copy()
indicator1_results.loc[indicator1_results.index[0], 'Core Portfolio Value'] = 100
indicator1_results.loc[indicator1_results.index[0], 'Selection Portfolio Value'] = 100
indicator1_results.loc[datetime.datetime.strptime('1978-12-31', '%Y-%m-%d').date()] = [0,0, 100,100]  # adding a row
indicator1_results = indicator1_results.sort_index()
for i in range(indicator1_results.index.size - 1):
    indicator1_results['Core Portfolio Value'].iloc[i+1] = (indicator1_results['Russell 1000 Core'].apply(percentagecompound).iloc[i+1] * indicator1_results['Core Portfolio Value'].iloc[i])
    indicator1_results['Selection Portfolio Value'].iloc[i+1] = (indicator1_results['Selection Return'].apply(percentagecompound).iloc[i+1] * indicator1_results['Selection Portfolio Value'].iloc[i])

indicator1_results
    

In [ ]:
#Creates Stats DF

indicator_Ret = ((indicator_pick[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].apply(percentagecompound).prod() ** (1/time_period) - 1)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Return'})
indicator_Std = (indicator_pick[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].std().apply(annualfromquarterlystd)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Std'})
indicator_Stats = pd.merge(indicator_Ret, indicator_Std, on='Strategy', how = 'left')
indicator_Stats['Sharpe'] = ((indicator_Stats['Annualized Return'] - 2) / indicator_Stats['Annualized Std']).round(2)
indicator_Stats = indicator_Stats.set_index('Strategy')
indicator_Stats


In [ ]:
#Plots selected portfolio vs Russell 1000 Core over period. 

fig = px.line(indicator1_results.reset_index(), x = 'date', y=['Core Portfolio Value','Selection Portfolio Value'], title='Core vs Indicator Selection v1')
fig.show()

In [ ]:
#Value of $100 DF

indicator1_results[['Core Portfolio Value','Selection Portfolio Value']].iloc[-1].round(2).reset_index().rename(columns = {'index':'Portfolio', 0:'Ending Port Val'}).set_index('Portfolio').style.set_caption('Value of $100')


In [ ]:
# creates trade schedule

indicator_trade_pivot = indicator_pick[['Trade/Hold','Favored']].reset_index()
indicator_trade_pivot.loc[indicator_trade_pivot['Trade/Hold'] == 'Trade', 'Trade/Hold' ] = indicator_trade_pivot['Trade/Hold'] + " - " + indicator_trade_pivot['Favored']#momentum_2_trade_pivot = momentum_2_trade_pivot.drop(columns = 'Pick')
indicator_trade_pivot = indicator_trade_pivot.drop(columns = 'Favored')
indicator_trade_pivot['Month'] = indicator_trade_pivot['date'].astype(str).str.split('-').str[1]
indicator_trade_pivot['Year'] = indicator_trade_pivot['date'].astype(str).str.split('-').str[0]
indicator_trade_pivot = indicator_trade_pivot.drop(columns = 'date').pivot(index = 'Year', columns = 'Month').rename(columns = {'03': 'Qtr 1','06': 'Qtr 2','09': 'Qtr 3','12': 'Qtr 4'}).style.apply(lambda x: ["background: green" if 'Trade' in value else "" for value in x], axis = 1)
indicator_trade_pivot


In [ ]:
#Creates Trade data frame

indicator_trades = indicator_pick[['Russell 1000 Core','Trade/Hold']].groupby('Trade/Hold').count().rename(columns = {'Russell 1000 Core':'Total'})
indicator_trades['Average Yearly Trades'] = indicator_trades['Total'] / time_period
indicator_trades['Average Yearly Trades'] = indicator_trades['Average Yearly Trades'].round(1)
indicator_trades = indicator_trades.drop('X')
indicator_trades


# Plots

In [ ]:
# Plots all 3 portfolio returns

subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(simple_yearly_md.apply(hundred).reset_index(), x="date", y=["Russell 1000 Core",'Russell 1000 Growth','Russell 1000 Value'],
            barmode='group',
             height=400, 
            ) 
subfig.update_layout(title= 'Yearly Returns (Pct)')
#fig.update_traces(line_color='#0000ff', line_width=5)
#fig2.update_traces(yaxis="y2",line_color='#0F0A09')
subfig.add_traces(fig.data)
subfig.update_xaxes(nticks=37)
subfig.update_xaxes(tickangle=90)
subfig.show()

In [ ]:
# Plots all 3 portfolio returns vs Inflation

subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(simple_yearly_md.apply(hundred).reset_index(), x="date", y=["Russell 1000 Core",'Russell 1000 Growth','Russell 1000 Value'],
            barmode='group',
             height=400) 
fig2 = px.line(simple_yearly_md.reset_index(),
        x='date',
        y='Inflation Change')

subfig.update_layout(title= 'Yearly Returns (Pct) vs Changes in Inflation', yaxis_title="Returns",)
#fig.update_traces(line_color='#0000ff', line_width=5)
fig2.update_traces(yaxis="y2",line_color='#0F0A09')
subfig.add_traces(fig.data + fig2.data)
subfig.update_xaxes(nticks=20)
subfig.update_xaxes(tickangle=45)
subfig.update_yaxes(title_text="Inflation", secondary_y=True)
subfig.show()

In [ ]:
# Plots all 3 portfolio returns vs Inflation

subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(simple_yearly_md.apply(hundred).reset_index(), x="date", y=["Russell 1000 Core",'Russell 1000 Growth','Russell 1000 Value'],
            barmode='group',
             height=400) 
fig2 = px.line(simple_yearly_md.reset_index(),
        x='date',
        y='Fed Rate Change')

subfig.update_layout(title= 'Yearly Returns (Pct) vs Changes in Fed Rate', yaxis_title="Returns",)
#fig.update_traces(line_color='#0000ff', line_width=5)
fig2.update_traces(yaxis="y2",line_color='#0F0A09')
subfig.add_traces(fig.data + fig2.data)
subfig.update_xaxes(nticks=20)
subfig.update_xaxes(tickangle=45)
subfig.update_yaxes(title_text="Fed Rate Change", secondary_y=True)
subfig.show()

In [ ]:
# Plots all 3 portfolio returns vs Inflation

subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(simple_yearly_md.apply(hundred).reset_index(), x="date", y=["Russell 1000 Core",'Russell 1000 Growth','Russell 1000 Value'],
            barmode='group',
             height=400) 
fig2 = px.line(simple_yearly_md.reset_index(),
        x='date',
        y='Unemployment Change')

subfig.update_layout(title= 'Yearly Returns (Pct) vs Changes in Unemployment', yaxis_title="Returns",)
#fig.update_traces(line_color='#0000ff', line_width=5)
fig2.update_traces(yaxis="y2",line_color='#0F0A09')
subfig.add_traces(fig.data + fig2.data)
subfig.update_xaxes(nticks=20)
subfig.update_xaxes(tickangle=45)
subfig.update_yaxes(title_text="Unemployment Change", secondary_y=True)
subfig.show()

# Add Leading Indicators

# Renames Housing Start Column Headers

housing_starts = housing_start_df.rename(columns = {'observation_date':'date', 'HOUST2FNSA':'Housing Starts'})

#housing_starts

In [ ]:
#Creates Month, Day, Year Columns, and converts to numbers. 
housing_starts_df['Month'] = housing_starts_df['date'].astype(str).str.split('-').str[1]
housing_starts_df['Day'] = housing_starts_df['date'].astype(str).str.split('-').str[2]
housing_starts_df['YEAR'] = housing_starts_df['date'].astype(str).str.split('-').str[0]
housing_starts_df[['Month', 'YEAR', 'Day']] = housing_starts_df[['Month', 'YEAR', 'Day']].apply(pd.to_numeric)

# Adds century to YY
#man_emp_df.loc[man_emp_df['YEAR'] + 2000 > currentYear, "YEAR"] = man_emp_df['YEAR'] + 1900
#man_emp_df.loc[man_emp_df['YEAR'] + 2000 <= currentYear, "YEAR"] = man_emp_df['YEAR'] + 2000

#Calculates Change and Percentage Change
housing_starts_df['Housing Change'] =  housing_starts_df['Housing Starts'].shift(-1) - housing_starts_df['Housing Starts']
housing_starts_df['Housing Change (Pct)'] = (housing_starts_df['Housing Change'] / housing_starts_df['Housing Starts'])*100

#Sets index and removes unwanted columns / rows. 
housing_starts_df = housing_starts_df.set_index('YEAR')
housing_starts_df = housing_starts_df.drop(columns = ['date','Day'])
housing_starts_df = housing_starts_df[(housing_starts_df.index > 1978) & (housing_starts_df.index < 2016)]

housing_starts_df


In [ ]:
# Pull Consumer Confidence Index data for US Only

cci_df = cci_df.loc[cci_df['LOCATION'] == 'USA'].reset_index().drop(columns = 'index')

#cci_df

In [ ]:
# Pull only necessary columns and rename. 

cci_df = cci_df[['TIME','Value']].rename(columns = {'TIME':'date'})

#cci_df

In [ ]:
#Creates Month, Day, Year Columns, and converts to numbers. 
cci_df['Month'] = cci_df['date'].str.split('-').str[1]
cci_df['Day'] = cci_df['date'].str.split('-').str[2]
cci_df['YEAR'] = cci_df['date'].str.split('-').str[0]
cci_df[['Month', 'YEAR', 'Day']] = cci_df[['Month', 'YEAR', 'Day']].apply(pd.to_numeric)

#Calculates Change and Percentage Change
cci_df['CCI Change'] =  cci_df['Value'].shift(-1) - cci_df['Value']
cci_df['CCI Change (Pct)'] = (cci_df['CCI Change'] / cci_df['Value'])*100

#Sets index and removes unwanted columns / rows. 
cci_df = cci_df.set_index('YEAR')
cci_df = cci_df.drop(columns = ['date','Day'])
cci_df = cci_df[(cci_df.index > 1978) & (cci_df.index < 2016)]

cci_df

In [ ]:
#Creates Month, Day, Year Columns, and converts to numbers. 
man_emp_df['Month'] = man_emp_df['date'].astype(str).str.split('-').str[1]
man_emp_df['Day'] = man_emp_df['date'].astype(str).str.split('-').str[2]
man_emp_df['YEAR'] = man_emp_df['date'].astype(str).str.split('-').str[0]
man_emp_df[['Month', 'YEAR', 'Day']] = man_emp_df[['Month', 'YEAR', 'Day']].apply(pd.to_numeric)

# Adds century to YY
#man_emp_df.loc[man_emp_df['YEAR'] + 2000 > currentYear, "YEAR"] = man_emp_df['YEAR'] + 1900
#man_emp_df.loc[man_emp_df['YEAR'] + 2000 <= currentYear, "YEAR"] = man_emp_df['YEAR'] + 2000

#Calculates Change and Percentage Change
man_emp_df['Man Emp Change'] =  man_emp_df['MANEMP'].shift(-1) - man_emp_df['MANEMP']
man_emp_df['Man Emp Change (Pct)'] = (man_emp_df['Man Emp Change'] / man_emp_df['MANEMP'])*100

#Sets index and removes unwanted columns / rows. 
man_emp_df = man_emp_df.set_index('YEAR')
man_emp_df = man_emp_df.drop(columns = ['date','Day'])
man_emp_df = man_emp_df[(man_emp_df.index > 1978) & (man_emp_df.index < 2016)]

man_emp_df

In [ ]:
#Creates Merged Frame off CCI, Housing Starts, and Manufacturing employment data. 

leading_merge = pd.merge(cci_df,housing_starts_df, on = ['YEAR', 'Month'], how = 'left').reset_index()
leading_merge = pd.merge(leading_merge,man_emp_df, on = ['YEAR', 'Month'], how = 'left')

#Creates Quarter Column and sets index. 
leading_merge['Quarter'] = leading_merge['Month'].apply(quartercalc)
leading_merge = leading_merge.set_index(['YEAR','Quarter','Month'])

leading_merge

## Monthly - Leading

In [ ]:
# Drops all Value and Pct Columns, as well as quarter to make monthly table. 

monthly_leading_join = leading_merge.reset_index().drop(columns = ['Quarter','Value','CCI Change (Pct)','Housing Starts','Housing Change (Pct)', 'MANEMP','Man Emp Change (Pct)'])
monthly_strat_join = market_merge_df_compound.reset_index().drop(columns = ['Quarter','Fed Rate Change (Pct)','Unemployment Change (Pct)', 'Inflation Change (Pct)']).set_index(['YEAR','Month','date']).apply(percentreverse).reset_index()
simple_leading_monthly_md = pd.merge(monthly_strat_join,monthly_leading_join, how = 'left', on = ['YEAR','Month']).drop(columns = ['YEAR','Month']).set_index('date')

simple_leading_monthly_md


In [ ]:
# Plts heatmap showing leading data correlation. 

market_data_corr = simple_leading_monthly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True).set_title('Leading Correlation based on Monthly Values (Simple Indicator)')
plt.show()

## Quarterly - Leading

In [ ]:
# Creates Quarterly Leading MD table

quarterly_leading_join = leading_merge[['CCI Change','Housing Change','Man Emp Change']].reset_index().drop(columns = 'Month').groupby(['YEAR','Quarter']).sum().reset_index()
quarterly_strat_join = market_merge_df_compound.reset_index().drop(columns = ['Month','Fed Rate Change (Pct)','Unemployment Change (Pct)', 'Inflation Change (Pct)']).set_index(['YEAR','Quarter']).groupby(['YEAR','Quarter']).agg({'date':'max','Russell 1000 Core':'prod','Russell 1000 Growth':'prod','Russell 1000 Value':'prod'}).reset_index().set_index(['YEAR','Quarter','date']).apply(percentreverse).reset_index()
simple_leading_quarterly_md = pd.merge(quarterly_strat_join,quarterly_leading_join, how = 'left', on = ['YEAR','Quarter']).drop(columns = ['YEAR','Quarter']).set_index('date')

simple_leading_quarterly_md


In [ ]:
# Plts heatmap showing quarterly leading data correlation. 

market_data_corr = simple_leading_quarterly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True).set_title('Leading Correlation based on Quarterly Values (Simple Indicator)')
plt.show()

## Yearly - Leading

In [ ]:
# Creates Quarterly Leading MD table

yearly_leading_join = leading_merge[['CCI Change','Housing Change','Man Emp Change']].reset_index().drop(columns = ['Month', 'Quarter']).groupby(['YEAR']).sum().reset_index()
yearly_strat_join = market_merge_df_compound.reset_index().drop(columns = ['Month','Quarter','Fed Rate Change (Pct)','Unemployment Change (Pct)', 'Inflation Change (Pct)']).set_index(['YEAR']).groupby(['YEAR']).agg({'date':'max','Russell 1000 Core':'prod','Russell 1000 Growth':'prod','Russell 1000 Value':'prod'}).reset_index().set_index(['YEAR','date']).apply(percentreverse).reset_index()
simple_leading_yearly_md = pd.merge(yearly_strat_join,yearly_leading_join, how = 'left', on = ['YEAR']).drop(columns = ['YEAR']).set_index('date')

simple_leading_yearly_md


In [ ]:
# Plts heatmap showing yearly leading data correlation. 

market_data_corr = simple_leading_yearly_md.reset_index().drop(columns = ['date']).corr()
correlation = market_data_corr
sn.heatmap(correlation, annot=True).set_title('Leading Correlation based on Yearly Values (Simple Indicator)')
plt.show()

# Factor Pick #2 (Leading)

In [ ]:
#Creates base model table from quarterly leading MD table
indicator_leading = simple_leading_quarterly_md.copy()

In [ ]:
# Assigns values to "Flags" used to determine which portfolio to use. 
# 1 = Value Weighted
# - 1 = Growth Weighted
# 0 = uncertain

indicator_leading.loc[(indicator_leading['CCI Change']<0),'CCI Flag'] = 1
indicator_leading.loc[(indicator_leading['CCI Change']>0),'CCI Flag'] = -1
indicator_leading.loc[(indicator_leading['CCI Change']==0),'CCI Flag'] = 0
indicator_leading.loc[(indicator_leading['Housing Change']<0),'Housing Flag'] = -1
indicator_leading.loc[(indicator_leading['Housing Change']>0),'Housing Flag'] = 2 #Adding additional weight for high correlation
indicator_leading.loc[(indicator_leading['Housing Change']==0),'Housing Flag'] = 0
indicator_leading.loc[(indicator_leading['Man Emp Change']<0),'Man Emp Flag'] = -1
indicator_leading.loc[(indicator_leading['Man Emp Change']>0),'Man Emp Flag'] = 2
indicator_leading.loc[(indicator_leading['Man Emp Change']==0),'Man Emp Flag'] = 0


In [ ]:
#Sums Flags to create a prediction
#Positive numbers and 0s will use the Value weighted portfolio.
#Negative numbers will use growth weighted. 

indicator_leading['Flag_Sum'] = indicator_leading['CCI Flag'] + indicator_leading['Housing Flag'] + indicator_leading['Man Emp Flag']
indicator_leading


In [ ]:
# Assigns favored portfolios based on pick column.

indicator_leading.loc[(indicator_leading['Flag_Sum'].shift() >0),'Favored'] = 'Russell 1000 Value'
indicator_leading.loc[(indicator_leading['Flag_Sum'].shift() ==0),'Favored'] = 'Russell 1000 Value'
indicator_leading.loc[(indicator_leading['Flag_Sum'].shift() <0),'Favored'] = 'Russell 1000 Growth'
indicator_leading = indicator_leading.replace(np.nan, 'Russell 1000 Value')
indicator_leading

In [ ]:
# Checks to see which portfolio was held last quarter and determines if a Trade is necessary. 

indicator_leading.loc[indicator_leading['Favored'] != indicator_leading['Favored'].shift(), "Trade/Hold"] = "Trade"
indicator_leading = indicator_leading.replace(np.nan, 'X')
indicator_leading


In [ ]:
# Assigns Portfolio weights to model
indicator_leading['Value Weighted Return'] = indicator_leading['Russell 1000 Core'] * .02 + indicator_leading['Russell 1000 Value'] * .97 + indicator_leading['Russell 1000 Growth'] * .01
indicator_leading['Growth Weighted Return'] = indicator_leading['Russell 1000 Core'] * .1 + indicator_leading['Russell 1000 Growth'] * .2 + indicator_leading['Russell 1000 Value'] * .7
indicator_leading['Core Weighted Return'] = indicator_leading['Russell 1000 Core'] * .2 + indicator_leading['Russell 1000 Value'] * .8 + indicator_leading['Russell 1000 Growth'] * .0

# Creates rows with performance of all 3 strategies
indicator_leading.loc[indicator_leading['Favored'] == 'Russell 1000 Core', "Selection Return"] =  indicator_leading['Value Weighted Return']
indicator_leading.loc[indicator_leading['Favored'] == 'Russell 1000 Growth', "Selection Return"] =  indicator_leading['Growth Weighted Return']
indicator_leading.loc[indicator_leading['Favored'] == 'Russell 1000 Value', "Selection Return"] =  indicator_leading['Value Weighted Return']

indicator_leading['Favored'] = indicator_leading['Favored'].apply(portswitch)

indicator_leading


In [ ]:
#Generates results table and calculates value of $100

indicator_leading_results = indicator_leading[['Russell 1000 Core','Selection Return']].copy()
indicator_leading_results.loc[indicator_leading_results.index[0], 'Core Portfolio Value'] = 100
indicator_leading_results.loc[indicator_leading_results.index[0], 'Selection Portfolio Value'] = 100
indicator_leading_results.loc[datetime.datetime.strptime('1978-12-31', '%Y-%m-%d').date()] = [0,0, 100,100]  # adding a row
indicator_leading_results = indicator_leading_results.sort_index()
for i in range(indicator_leading_results.index.size - 1):
    indicator_leading_results['Core Portfolio Value'].iloc[i+1] = (indicator_leading_results['Russell 1000 Core'].apply(percentagecompound).iloc[i+1] * indicator_leading_results['Core Portfolio Value'].iloc[i])
    indicator_leading_results['Selection Portfolio Value'].iloc[i+1] = (indicator_leading_results['Selection Return'].apply(percentagecompound).iloc[i+1] * indicator_leading_results['Selection Portfolio Value'].iloc[i])

indicator_leading_results


In [ ]:
#Generates results table and calculates value of $100

indicator_leading_trade_view = indicator_leading[['Russell 1000 Core','Russell 1000 Value','Russell 1000 Growth','Selection Return','Favored','Trade/Hold']].copy()
indicator_leading_trade_view.loc[indicator_leading_trade_view.index[0], 'Core Portfolio Value'] = 100
indicator_leading_trade_view.loc[indicator_leading_trade_view.index[0], 'Selection Portfolio Value'] = 100
indicator_leading_trade_view.loc[datetime.datetime.strptime('1978-12-31', '%Y-%m-%d').date()] = [0,0, 0,0,'Value Weighted','X',100,100]  # adding a row
indicator_leading_trade_view = indicator_leading_trade_view.sort_index()
for i in range(indicator_leading_trade_view.index.size - 1):
    indicator_leading_trade_view['Core Portfolio Value'].iloc[i+1] = (indicator_leading_trade_view['Russell 1000 Core'].apply(percentagecompound).iloc[i+1] * indicator_leading_trade_view['Core Portfolio Value'].iloc[i])
    indicator_leading_trade_view['Selection Portfolio Value'].iloc[i+1] = (indicator_leading_trade_view['Selection Return'].apply(percentagecompound).iloc[i+1] * indicator_leading_trade_view['Selection Portfolio Value'].iloc[i])

# Calculates the beginning index weights of portfolio based on 
indicator_leading_trade_view.loc[indicator_leading_trade_view['Favored'] == 'Value Weighted' , "Beginning Value Allocation"] =  indicator_leading_trade_view['Selection Portfolio Value'].shift() * .97
indicator_leading_trade_view.loc[indicator_leading_trade_view['Favored'] == 'Value Weighted' , "Beginning Core Allocation"] =  indicator_leading_trade_view['Selection Portfolio Value'].shift() * .02
indicator_leading_trade_view.loc[indicator_leading_trade_view['Favored'] == 'Value Weighted' , "Beginning Growth Allocation"] =  indicator_leading_trade_view['Selection Portfolio Value'].shift() * .01
indicator_leading_trade_view.loc[indicator_leading_trade_view['Favored'] == 'Growth Weighted' , "Beginning Value Allocation"] =  indicator_leading_trade_view['Selection Portfolio Value'].shift() * .7
indicator_leading_trade_view.loc[indicator_leading_trade_view['Favored'] == 'Growth Weighted' , "Beginning Core Allocation"] =  indicator_leading_trade_view['Selection Portfolio Value'].shift() * .1
indicator_leading_trade_view.loc[indicator_leading_trade_view['Favored'] == 'Growth Weighted' , "Beginning Growth Allocation"] =  indicator_leading_trade_view['Selection Portfolio Value'].shift() * .2

#Calculate ending portfolio allocations, by multiplying weights against ending portfolio value for the month

indicator_leading_trade_view['Ending Value Allocation'] = indicator_leading_trade_view['Beginning Value Allocation'] * (indicator_leading_trade_view['Russell 1000 Value'].apply(percentagecompound))
indicator_leading_trade_view['Ending Core Allocation'] = indicator_leading_trade_view['Beginning Core Allocation'] * (indicator_leading_trade_view['Russell 1000 Core'].apply(percentagecompound))
indicator_leading_trade_view['Ending Growth Allocation'] = indicator_leading_trade_view['Beginning Growth Allocation'] * (indicator_leading_trade_view['Russell 1000 Growth'].apply(percentagecompound))

#Calculates beginning percent by dividing Beginning Value Allocation and last months selection portfolio value - Should always equal weightings of the created portfolios. 

indicator_leading_trade_view['Beginning Value Percent'] = indicator_leading_trade_view['Beginning Value Allocation'] / indicator_leading_trade_view['Selection Portfolio Value'].shift()
indicator_leading_trade_view['Beginning Core Percent'] = indicator_leading_trade_view['Beginning Core Allocation'] / indicator_leading_trade_view['Selection Portfolio Value'].shift()
indicator_leading_trade_view['Beginning Growth Percent'] = indicator_leading_trade_view['Beginning Growth Allocation'] / indicator_leading_trade_view['Selection Portfolio Value'].shift()

#Determines ending value percentages by dividing end allocation and the selection portfolio value

indicator_leading_trade_view['Ending Value Percent'] = (indicator_leading_trade_view['Ending Value Allocation'] / indicator_leading_trade_view['Selection Portfolio Value']).round(4)
indicator_leading_trade_view['Ending Core Percent'] = (indicator_leading_trade_view['Ending Core Allocation'] / indicator_leading_trade_view['Selection Portfolio Value']).round(4)
indicator_leading_trade_view['Ending Growth Percent'] = (indicator_leading_trade_view['Ending Growth Allocation'] / indicator_leading_trade_view['Selection Portfolio Value']).round(4)

# Determines trade size and value in pct. 

indicator_leading_trade_view['Value Trade'] = (indicator_leading_trade_view['Beginning Value Percent'] - indicator_leading_trade_view['Ending Value Percent'].shift()).round(4)
indicator_leading_trade_view['Core Trade'] = (indicator_leading_trade_view['Beginning Core Percent'] - indicator_leading_trade_view['Ending Core Percent'].shift()).round(4)
indicator_leading_trade_view['Growth Trade'] = (indicator_leading_trade_view['Beginning Growth Percent'] - indicator_leading_trade_view['Ending Growth Percent'].shift()).round(4)
indicator_leading_trade_view = indicator_leading_trade_view.replace(np.nan,0)

# Updates Trade columns to directional strings. 

indicator_leading_trade_view['Value Trade'] = indicator_leading_trade_view['Value Trade'].apply(advtradeflag)
indicator_leading_trade_view['Core Trade'] = indicator_leading_trade_view['Core Trade'].apply(advtradeflag)
indicator_leading_trade_view['Growth Trade'] = indicator_leading_trade_view['Growth Trade'].apply(advtradeflag)

pd.set_option('display.max_rows',20)
pd.set_option('display.max_columns',20)

#indicator_leading_results = indicator_leading_results['']

indicator_leading_trade_view.loc[((indicator_leading_trade_view['Value Trade'] == 'Do Not Trade') & (indicator_leading_trade_view['Core Trade'] == 'Do Not Trade') & (indicator_leading_trade_view['Growth Trade'] == 'Do Not Trade')), 'NewTradeFlag'] = 'X'
#indicator_leading_trade_view[1:]
#indicator_leading_trade_view = indicator_leading_trade_view.loc[indicator_leading_trade_view['Trade/Hold'] == 'Trade']
indicator_leading_trade_view['NewTradeFlag'] = indicator_leading_trade_view['NewTradeFlag'].replace(np.nan,'Trade')
indicator_leading_trade_view = indicator_leading_trade_view[['Favored','Trade/Hold','NewTradeFlag','Beginning Value Percent','Beginning Core Percent','Beginning Growth Percent','Ending Value Percent','Ending Core Percent','Ending Growth Percent','Value Trade','Core Trade','Growth Trade']][1:]
indicator_leading_trade_view = indicator_leading_trade_view.loc[(indicator_leading_trade_view['Trade/Hold'] == 'Trade') & (indicator_leading_trade_view['NewTradeFlag'] == 'Trade')].drop(columns = 'NewTradeFlag')
indicator_leading_trade_view


In [ ]:
indicator_leading_Ret = ((indicator_leading[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].apply(percentagecompound).prod() ** (1/time_period) - 1)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Return'})
indicator_leading_Std = (indicator_leading[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].std().apply(annualfromquarterlystd)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Std'})
indicator_leading_Stats = pd.merge(indicator_leading_Ret, indicator_leading_Std, on='Strategy', how = 'left')
indicator_leading_Stats['Sharpe'] = ((indicator_leading_Stats['Annualized Return'] - 2) / indicator_leading_Stats['Annualized Std']).round(2)
indicator_leading_Stats = indicator_leading_Stats.set_index('Strategy')
indicator_leading_Stats


In [ ]:
#Calculates Annualized Return

((indicator_leading[['Russell 1000 Core','Russell 1000 Growth','Russell 1000 Value','Selection Return']].apply(percentagecompound).prod() ** (1/time_period) - 1)*100).reset_index().rename(columns = {'index': 'Strategy',0:'Annualized Return'})


In [ ]:
#Calculates number of trades over period

indicator_leading_trades = indicator_leading[['Russell 1000 Core','Trade/Hold']].groupby('Trade/Hold').count().rename(columns = {'Russell 1000 Core':'Total'})
indicator_leading_trades['Average Yearly Trades'] = indicator_leading_trades['Total'] / time_period
indicator_leading_trades['Average Yearly Trades'] = indicator_leading_trades['Average Yearly Trades'].round(1)
indicator_leading_trades = indicator_leading_trades.drop('X')
indicator_leading_trades


In [ ]:
#Plots selected portfolio vs Russell 1000 Core over period. 

fig = px.line(indicator_leading_results.reset_index(), x = 'date', y=['Core Portfolio Value','Selection Portfolio Value'], title='Core vs Indicator Selection v2')
fig.show()

In [ ]:
# Value of $100 DF

indicator_leading_results[['Core Portfolio Value','Selection Portfolio Value']].iloc[-1].round(2).reset_index().rename(columns = {'index':'Portfolio', 0:'Ending Port Val'}).set_index('Portfolio').style.set_caption('Value of $100')


In [ ]:
# Trade Schedule 

indicator_leading_trade_pivot = indicator_leading[['Trade/Hold','Favored']].reset_index()
indicator_leading_trade_pivot.loc[indicator_leading_trade_pivot['Trade/Hold'] == 'Trade', 'Trade/Hold' ] = indicator_leading_trade_pivot['Trade/Hold'] + " - " + indicator_leading_trade_pivot['Favored']#momentum_2_trade_pivot = momentum_2_trade_pivot.drop(columns = 'Pick')
indicator_leading_trade_pivot = indicator_leading_trade_pivot.drop(columns = 'Favored')
indicator_leading_trade_pivot['Month'] = indicator_leading_trade_pivot['date'].astype(str).str.split('-').str[1]
indicator_leading_trade_pivot['Year'] = indicator_leading_trade_pivot['date'].astype(str).str.split('-').str[0]
indicator_leading_trade_pivot = indicator_leading_trade_pivot.drop(columns = 'date').pivot(index = 'Year', columns = 'Month').rename(columns = {'03': 'Qtr 1','06': 'Qtr 2','09': 'Qtr 3','12': 'Qtr 4'}).style.apply(lambda x: ["background: green" if 'Trade' in value else "" for value in x], axis = 1)
indicator_leading_trade_pivot
